## Downloading Data from OSM

The open street map data is given in XML format. We only care about the [power](https://openinframap.org/#14.43/26.06828/-80.20208) categories, especially generators, substations, lines, transformers, and switches. Here we map the [Ft. Lauderdale Power Plant](https://openinframap.org/#14.43/26.06828/-80.20208), which consists of three power-plants and a few substations. The raw XML data can be downloaded from the [OSM website](https://www.openstreetmap.org/export#map=16/26.0720/-80.2004), some is shown below. 

In [ ]:
<!-- GENERATION -->
<node id="9816126697" lat="26.0850857" lon="-80.1251280" version="1" timestamp="2022-06-14T01:50:53Z" changeset="122348291" uid="1443767" user="Olyon">
    <tag k="generator:method" v="combustion"/>
    <tag k="generator:output:electricity" v="296 MW"/>
    <tag k="generator:source" v="gas"/>
    <tag k="generator:type" v="gas_turbine"/>
    <tag k="power" v="generator"/>
    <tag k="source" v="U.S. Energy Information Administration - EIA-860 - 2020"/>
    <tag k="start_date" v="2016-04"/>
  </node>
  <!-- POWER PLANT -->
<way id="542155046" version="14" timestamp="2023-02-26T01:59:19Z" changeset="133027405" uid="10661773" user="Local-Mapper">
    <nd ref="4196305262"/>
    <nd ref="5241799909"/>
<!-- many taken out for simplicity -->
    <nd ref="4196305262"/>
    <tag k="addr:city" v="Ft. Lauterdale"/>
    <tag k="addr:housenumber" v="8100"/>
    <tag k="addr:postcode" v="33316"/>
    <tag k="addr:state" v="FL"/>
    <tag k="addr:street" v="Eisenhower Boulevard"/>
    <tag k="landuse" v="industrial"/>
    <tag k="name" v="Port Everglades Power Plant"/>
    <tag k="operator" v="Florida Power &amp; Light"/>
    <tag k="plant:method" v="combustion"/>
    <tag k="plant:output:electricity" v="1352 MW"/>
    <tag k="plant:source" v="gas"/>
    <tag k="plant:type" v="combined_cycle"/>
    <tag k="power" v="plant"/>
    <tag k="ref:US:EIA" v="617"/>
    <tag k="source" v="U.S. Energy Information Administration - EIA-860 - 2020"/>
    <tag k="start_date" v="1960-06"/>
    <tag k="wikidata" v="Q11996062"/>
  </way>
<!-- SUBSTATION -->
 <way id="48829189" visible="true" version="8" changeset="149247712" timestamp="2024-03-28T01:07:31Z" user="Olyon" uid="1443767">
  <nd ref="619471506"/>
  <nd ref="619471479"/>
<!-- many taken out for simplicity -->
  <nd ref="619471506"/>
  <tag k="frequency" v="60"/>
  <tag k="location" v="outdoor"/>
  <tag k="name" v="Lauderdale Plant Substation"/>
  <tag k="operator" v="Florida Power &amp; Light"/>
  <tag k="operator:wikidata" v="Q1430055"/>
  <tag k="power" v="substation"/>
  <tag k="substation" v="transmission"/>
  <tag k="voltage" v="230000;138000;13800"/>
 </way>
<!-- POWER LINE -->
  <way id="48467928" version="6" timestamp="2023-01-28T08:56:56Z" changeset="131799659" uid="13929699" user="Gamer2000">
    <nd ref="3721597479"/>
    <nd ref="615886297"/>
<!-- again, many taken out for simplicity-->
    <nd ref="1340766482"/>
    <tag k="cables" v="3"/>
    <tag k="frequency" v="60"/>
    <tag k="name" v="Laudania Sub FPL to Port Everglades Sub FPL 230KV"/>
    <tag k="operator" v="Florida Power &amp; Light"/>
    <tag k="operator:wikidata" v="Q1430055"/>
    <tag k="power" v="line"/>
    <tag k="voltage" v="230000"/>
  </way>
<!--  WHICH IS A COLLECTION OF POWER TOWERS -->
<node id="615886339" lat="26.0674905" lon="-80.1293144" version="3" timestamp="2015-08-30T15:03:32Z" changeset="33684428" uid="160651" user="dcat">
    <tag k="power" v="tower"/>

In [73]:
import requests
import xml.etree.ElementTree as ET
import queue
from collections import defaultdict

# function to convert an integer to a string with the appropriate suffix
Ith = lambda i: str(i) + ("th" if (abs(i) % 100 in (11,12,13)) else ["th","st","nd","rd","th","th","th","th","th","th"][abs(i) % 10])

def get_OSM_power_data(bbox, round_to=7):
    """ converts all power related items from open street map to a dictionary
     INPUT: bbox is a tuple of 4 floats (left, bottom, right, top) needed to define rectangular bounding box
            round_to is an integer that specifies the number of decimal places to round the bounding box to

     OUTPUT: a tuple of DATA dictionary and INDICES dictionary
                DATA is a dictionary of all power related items in the bounding box, so that 
                  DATA[id] = {'osm_type': 'node', 'lat': lat, 'lon': lon, tags...}
                             {'osm_type': 'way', 'nodes': [node1, node2, ...],... tags}
                             {'osm_type': 'relation', 'members': {'nodes': [node1, node2, ...], 'ways': [way1, way2, ...]... },... }
                INDICES is a dictionary of all power tags with their indices in the dictionary, so that
                  INDICES['generator'] = [id1, id2, ...] corresponding to all 'k'='power', 'v'='generator' items
    
        Gets all open street map items in the bounding box using an API request.
      If the bounding box includes more than 50k points, it will split the bounding box into 4 quadrants and repeat.
      Only filters items that have a 'power' tag.
    """
    # store all power data in dictionary
    DATA = {}
    # keep track of all indices with each power tag 
    # to easily access all power plants, for example
    POWER_IDXS = defaultdict(list)
    # keep track of the type of each power tag. some things might have 
    # multiple types.
    POWER_TYPES = {}
    
    def parse_tag(child, id, tags, type_='node'):
        ''' understand which items are listed as multiple OSM types while parsing tags '''
        key = child.get('k')
        val = child.get('v')
        if key and val:
            tags[key] = val
            # keep track of all indexes with power tag, and if they are nodes or ways
            if key == 'power':
                POWER_IDXS[val].append(id)
                oldtype = POWER_TYPES.get(val, None)
                if oldtype is None:
                    POWER_TYPES[val] = {type_: 1}
                else:
                    POWER_TYPES[val][type_] = POWER_TYPES[val].get(type_,0) + 1
    
    bbox = tuple(map(lambda x: round(x, round_to), bbox))
    Q = queue.Queue()
    Q.put(bbox)
    nbboxes = 1

    nnodes = nways = nrels = 0
    while nbboxes:
        bbox_ = Q.get()
        nbboxes -= 1
        # api request for selected bounding box
        url = "https://api.openstreetmap.org/api/0.6/map?bbox="
        url += ",".join(map(str, bbox_))
        #print(url)
        request = requests.get(url)   # make the api request
        response = request.content    # get the (bytes) response
        try:   # parse the response into a dictionary
            root = ET.fromstring(response)
        except ET.ParseError:
            # we requested too many nodes, so now let's
            # split the bounding box into 4 quadrants
            left, bottom, right, top = bbox_
            mid_x = round((left + right) / 2,round_to)
            mid_y = round((bottom + top) / 2,round_to)
            # add 4 quadrants to the queue
            Q.put((left, bottom, mid_x, mid_y))  # bottom left
            Q.put((mid_x, bottom, right, mid_y)) # bottom right
            Q.put((left, mid_y, mid_x, top))     # top left
            Q.put((mid_x, mid_y, right, top))    # top right
            nbboxes += 4
            continue
        
        # add all power related items to dictionary for later use
        for element in root:
            
            # make sure element has a power tag
            save = False
            for child in element:
                if child.get('k') == 'power':
                    save = True
                    break
            if not save: continue
            
            id = int(element.get('id'))
            if element.tag == 'node':
                tags = {"osm_type": "node", "lat": element.get('lat'), "lon": element.get('lon')}
                for child in element:   # assume all tages are child.tag == 'tag'
                    if child.tag == 'tag':
                        parse_tag(child, id, tags)
                DATA[id] = tags
                nnodes += 1
            elif element.tag == 'way':
                tags = {'osm_type': "way", 'nodes': []}
                for child in element:
                    if child.tag == 'tag':
                        parse_tag(child, id, tags, 'way')
                    elif child.tag == 'nd':
                        tags['nodes'].append(child.get('ref'))
                DATA[id] = tags
                nways += 1
            elif element.tag == 'relation':
                tags = {"osm_type": "relation", 'members': {}}
                for member in element:
                    if member.tag == 'member':
                        key = member.get('type')+'s'
                        if key not in tags['members']:
                            tags['members'][key] = []
                        tags['members'][key].append(member.get('ref'))
                    elif member.tag == 'tag':
                        parse_tag(member, id, tags, 'relation')
                DATA[id] = tags
                nrels += 1
            else:
                print("Unknown element: ", element)
    print(f"Rectangle {bbox} has {nnodes} nodes, {nways} ways, and {nrels} relations.")
    for key in POWER_TYPES.keys():
        print(f"\t power {key} has "+", ".join([f"{v} {k}s" for k, v in POWER_TYPES[key].items()])+".")
    
    return DATA, POWER_IDXS

In [74]:
#bounding box coords around Ft. Lauderdale, FL, power plant
bottom = 26.045381
left = -80.219148
top = 26.087833
right = -80.169414

ft_lauderdale = (left, bottom, right, top)

print('Getting data for Ft. Lauderdale power plant in Florida')
Df, If = get_OSM_power_data(ft_lauderdale)

print('Getting data for Shiloh wind farm in California')
shiloh_wind = (-121.9452,38.0780,-121.7295,38.2452)
D, I = get_OSM_power_data(shiloh_wind)


Getting data for Ft. Lauderdale power plant in Florida
Rectangle (-80.219148, 26.045381, -80.169414, 26.087833) has 2603 nodes, 138 ways, and 0 relations.
	 power tower has 1081 nodes.
	 power portal has 39 nodes.
	 power pole has 1397 nodes.
	 power link has 5 nodes.
	 power transformer has 18 nodes.
	 power line has 125 ways.
	 power substation has 8 ways.
	 power generator has 10 nodes.
	 power switch has 53 nodes.
	 power plant has 4 ways.
	 power minor_line has 1 ways.
Getting data for Shiloh wind farm in California
Rectangle (-121.9452, 38.078, -121.7295, 38.2452) has 4049 nodes, 141 ways, and 8 relations.
	 power tower has 1649 nodes.
	 power generator has 528 nodes, 1 ways.
	 power portal has 18 nodes.
	 power pole has 1850 nodes.
	 power switch has 2 nodes.
	 power transformer has 2 nodes.
	 power line has 52 ways.
	 power substation has 12 ways.
	 power plant has 3 ways, 8 relations.
	 power minor_line has 73 ways.


We can see that power plants and substations can either be closed ways or relations that effectively function as a closed way. 
Generators can be either a closed way or a node. 
Connections between stations are either line, minor_line, or cable and are open ways spanned by towers, portals, and poles. 
They are connected to stations via link, transformer, and switch


In [70]:
for plant_idx in I['plant']:
    val = D[plant_idx]
    print(f"Plant {plant_idx} has tags of")
    for key in val.keys():
        print(f"\t{key}: {val[key]}")

for k in I.keys():
    print(f"Power tag {k} has {len(I[k])} items in the bounding box.")
    for idx in I[k]:
        print(f"\t{D[idx]}")
    print()


Plant 59171725 has tags of
	osm_type: way
	nodes: ['419806928', '733816377', '733816340', '733816332', '419806933', '419806934', '419806927', '419806928']
	landuse: industrial
	name: Lambie Energy Center
	operator: Gilroy Energy Center LLC
	plant:output:electricity: 47 MW
	plant:source: gas
	power: plant
	ref:US:EIA: 55626
	source: U.S. Energy Information Administration - EIA-860 - 2021
	start_date: 2003-01
Plant 436955529 has tags of
	osm_type: way
	nodes: ['4347970331', '4347970346', '4347970310', '4347970337', '4347970338', '4347970331']
	landuse: industrial
	name: Creed Energy Center
	operator: Creed Energy Center LLC
	plant:output:electricity: 47 MW
	plant:source: gas
	power: plant
	ref:US:EIA: 55625
	source: U.S. Energy Information Administration - EIA-860 - 2021
	start_date: 2003-01
Plant 436955534 has tags of
	osm_type: way
	nodes: ['4347970352', '4347970292', '4347970351', '4347970296', '4347970344', '4347968682', '4347970352']
	landuse: industrial
	name: Goose Haven Energy Ce